# Final Project Code for Polynomai computation on GPU

In [1]:
from numba import cuda
from numba import float32,int32,int64
import numpy as np
import pdb
from math import floor
from decimal import *

# Sympy stuff for checking
from sympy import Poly, expand 
from sympy.abc import x, y, u, v


def parsePolynomialString(polyString):
    
    
    try:
        polyString = unicode(polyString,"utf-8")
    except TypeError: #check if already unicode
        pass
    
    repls = ('+', '!'), ('-', '!')
    temp = [l.strip() for l in reduce(lambda a, kv: a.replace(*kv), repls, polyString ).split('!')]
    temp2 = np.asarray([split_coefficient_variable(t) for t in temp])
    temp2[:,0] = map(lambda x: u'1' if x==u'' else x,temp2[:,0])
    
    signs = ''.join(ch for ch in polyString if ch == '+' or ch == '-')
    if len(signs)==len(temp2)-1:
        signs = '+'+signs
    signs = np.asarray([c for c in signs])
    
    
    coeffs = [int(s+val) for (s,val) in zip(signs,temp2[:,0])]
    variables = temp2[:,1]
    
    return coeffs,variables

def split_coefficient_variable(string):
    n = len(string)
    for ind,c in enumerate(string):
        if c.isnumeric():
            if ind == n-1:
                return (string,u'0')
            else:
                continue
        else:
            return (string[0:ind],string[ind+1::])
        
def findDegreeUnivariate(v):
    end_ind = -1 # a hack way for a flag
    start_ind = -1
    for i,c in enumerate(v):
        if c.isnumeric() and start_ind == -1:
            start_ind = i
            if start_ind == len(v) -1:
                end_ind = start_ind
                break
        elif (not c.isnumeric() and end_ind == -1 and not start_ind  == -1) or i == len(v)-1:
            end_ind = i
            break

        else:
            continue
    #pdb.set_trace()
    if not start_ind == end_ind:
         if end_ind == 0 and start_ind == -1:
            return 1
         else:
            t = v[start_ind:end_ind+1]
       
    else:
        t = v[start_ind]
    
    return int(t)

def buildDegrees(varsfull):
    ''' Simple wrapper function that builds a list of list for all the degrees of each univariate variable '''
    degreeLists = [[findDegreeUnivariate(var) for var in vars.split()] for vars in varsfull]
    return degreeLists

def addZeros(pVars,pCoef,maxDegree,case='univariate'):
    if case=='univariate':
        cnt = maxDegree
        res = np.array([],dtype = np.int32)
        for i,(v,c) in enumerate(zip(pVars,pCoef)):
            if v==u'0':
                temp =0 
            else:
                temp = v.split('^')[-1]

            temp = int(temp)
            #pdb.set_trace()
            if not temp ==cnt:
                # traditonal zero imputation    
                diff = cnt-temp
                res = np.append(res,np.zeros(diff))
                cnt -= diff

            res = np.append(res,c)
            
            cnt -= 1
     
            if i == (len(pVars)-1) and cnt>=0:
                #pdb.set_trace()
                res=np.append(res,np.zeros(temp))

        
        return res
def primesfrom3to(n):
    """ Returns a array of primes, 3 <= p < n """
    sieve = np.ones(n/2, dtype=np.bool)
    for i in xrange(3,int(n**0.5)+1,2):
        if sieve[i/2]:
            sieve[i*i/2::i] = False
    return 2*np.nonzero(sieve)[0][1::]+1
    
def get_mod_primes(N,M):
    primes = primesfrom3to(N)
    primes = primes.tolist()
    total = 1L
    mvect = np.array([],dtype=np.int32)
    for p in primes:
        total *= p
        mvect = np.append(mvect,p)
        #print total
        if total > M:
            return mvect
    return mvect

def generate_evaluationpoints(degreeResult):
    return np.arange(1,degreeResult)


def newton_interp_univariate(x,b,d):
    # x: set of points
    # b: corresponding value evaluated at the point
    # d: degree of polynomial (should be equal to len(x)-1
    n = len(x)
    V = np.array([x**i for i in range(d+1)][::-1]).T
    A = np.dot(np.linalg.inv(V),b)
    return A


def EGCD(a,b):
    cc = a
    dd = b
    c1 = 1
    c2 =0
    d1 =0
    d2 = 1
    
    while not dd==0:
        try:
            q = cc/dd
        except TypeError:
            pdb.set_trace()
            
        r = cc - int(q*dd)
        r = int(r)
        cc = dd
        dd = r

        r1 = c1 - int(q*d1)
        r1 = int(r1)
        c1 = d1
        d1 = r1

        r2 = c2 - int(q*d2)
        r2 = int(r2)
        c2 = d2
        d2 = r2
    
    return (cc,c1,c2)

def cra_incremental(rvect,mvect):
    k = len(rvect)
    assert(len(rvect) == len(mvect))
    
    M = mvect[0]
    res = rvect[0]
    M_invs = []
    for i in range(1,k):
        M_inv = EGCD(M,mvect[i])[1] 
        if M_inv == 0: pdb.set_trace()
        M_invs.append(M_inv)
        c = int(M_inv%mvect[i])
        rprime = int(res%mvect[i])
        s = c*(rvect[i] - rprime)#%mvect[i]
        s = int(s)
        res = int(res+s*M)
        #import pdb; pdb.set_trace()
        M = int(M*mvect[i])
    return res

def MRC_alg(rvect,mvect,c):
    k = len(rvect)
    gamma = np.empty(k,dtype = np.int32)
    gamma[0] = rvect[0]
    #gamma[0] = rvect[0]
    M = np.ones(k,dtype=np.int32)

    for i in range(1,k):
        gamma[i] = ((rvect[i] - gamma[0])*c[i]) % mvect[i]
        M[i] = (mvect[0]*c[i]) % mvect[i]

    for i in range(1,k-1):
        for j in range(i+1,k):
            gamma[j] = (gamma[j] - gamma[i]*M[j]) % mvect[j]
            M[j] = (M[j]*mvect[i]) % mvect[j]
    return gamma,M

def homer_scheme(mvect,gammas,i):
    if i==len(mvect):
        return 1
    return gammas[i]+mvect[i]*(homer_scheme(mvect,gammas,i+1))

def symmetric_residue(r,m):
    if r <= m // 2: # divide by 2
        return r
    return r - m


def newton_interp_mod(uvect,alpha,c,p):
    k = len(uvect)
    gamma = np.empty(k,dtype=np.int32)
    gamma[0] = uvect[0]

    M = np.ones(k,dtype=np.int32)

    for i in range(1,k):
        gamma[i] = ((uvect[i] - gamma[0])*c[i]) % p
        M[i] = ((alpha[i]-alpha[0])*c[i]) % p

    for i in range(1,k-1):
        for j in range(i+1,k):
            gamma[j] = (gamma[j] - gamma[i]*M[j]) %p
            M[j] = (M[j]*(alpha[j]-alpha[i])) % p

    return gamma,M

def homer_scheme_newton(alphas,gammas,i):
    if i == len(alphas):
        return 1
    return gammas[i]+(alphas[-1]-alphas[i])*(homer_scheme_newton(alphas,gammas,i+1))

# p=5
# aa = [(alpha[-1] - alpha[i]) for i in range(len(alpha))]
# gamma2 = [0] + [EGCD(reduce(lambda x,y:x*y,aa[0:k_+1]),ai)[1] for (k_,ai) in enumerate(aa[1:])]
# vv = newton_interp_mod([0,3,4],[0,1,2],gamma2,p)[0]
# #G = reduce(lambda x,y: x*y,m)
# temp = homer_scheme(aa,vv,0)%p   
@cuda.jit('int32[:](int32[:],int32[:],int32[:],int32[:],int32[:])',device=True, inline = True)
def MRC_alg_Cuda(gamma,rvect,mvect,c,M):
    # initalizations
    k = len(rvect)
    gamma[0] = rvect[0]
    M[0] = 1
 
    for i in range(1,k):
        gamma[i] = ((rvect[i] - gamma[0])*c[i]) % mvect[i]
        M[i] = (mvect[0]*c[i]) % mvect[i]

    for i in range(1,k-1):
        for j in range(i+1,k):
            gamma[j] = (gamma[j] - gamma[i]*M[j]) % mvect[j]
            M[j] = (M[j]*mvect[i]) % mvect[j]
            
    return gamma

@cuda.jit(argtypes=[int32[:],int32[:],int32[:],int32[:]], target='gpu')
def CRA_kernel(a,res,m,c):
    sa = cuda.shared.array(shape=(256,), dtype=int32) # residues
    sb = cuda.shared.array(shape=(256,), dtype=int32) # gamma Mixed radix vector (returned from the MRC device function)
    sM = cuda.shared.array(shape=(256,), dtype=int32) # M coefficient vector
    sc = cuda.shared.array(shape=(256,), dtype=int32) # precomputed coefficient 
    sm = cuda.shared.array(shape=(256,), dtype=int32) # prime vector
    
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x
    
    i = cuda.grid(1)
    if i < len(a):
        sa[tx] = a[i]
        sc[tx] = c[tx]
        sm[tx] = m[tx]
        cuda.syncthreads()
        #sc[tx] = c[tx]
        #sM[tx] = 1
        if tx==0:
            os = bw
            sb[tx:os] = MRC_alg_Cuda(sb[tx:os],sa[tx:os],sm[tx:os],sc[tx:os],sM[tx:os])
            
            res[i:i+os] = sb[tx:os]

@cuda.jit(argtypes=[int32[:],int32[:],int32,int32,int32,int32,int32], target='gpu')
def evaluate_polynomial(a,c,n,d,k,l,alpha_os):
    i = cuda.grid(1)
    N = n*d*k*l
    #we need a representation for the polynomial (x^(p-1),x^p,...x^0)
    
    if i<N:

        alpha = (i%(n*d))/d
    
        degree = i%d
        ind = degree #it just so happens that the degree is equal to the index in this case
        offset = (i/(n*d)*d)
        #cuda.syncthreads() # ---_ myself for shared memory implemtation
        c[i] = a[ind+offset]*(alpha+alpha_os)**degree
        
        

## Infinite Precision Chinese Remaindering

In [2]:
    '''Lets take a look at some modular arthimitic'''
num = 515033918625080621034667968750231312512515215125215125125
print "num has {} digits ".format(len(str(num)))

mvect = get_mod_primes(10000,num).tolist()
print "{} primes were required to reconstruct num\n".format(len(mvect))
residues =[int(num%m) for m in mvect] 
c = [0] + [EGCD(reduce(lambda x,y:x*y,mvect[0:k+1]),m)[1] for (k,m) in enumerate(mvect[1:])]
vv,M = MRC_alg(residues,mvect,c)
vv = vv.tolist()
M = M.tolist()

print "v: {} \n".format(vv)

u = homer_scheme(mvect,vv,0)%reduce(lambda x,y: x*y,mvect)

mvect2 = get_mod_primes(10000,u).tolist()
print "Solution Residues: {} \n".format(residues)
residues2 =[int(u%m) for m in mvect2]
print "Check Residues: {}\n".format(residues)


num has 57 digits 
34 primes were required to reconstruct num

v: [2, 1, 0, 4, 4, 11, 3, 0, 11, 11, 3, 18, 42, 36, 50, 48, 12, 27, 4, 27, 6, 4, 59, 49, 50, 4, 68, 57, 19, 124, 46, 0, 119, 102] 

Solution Residues: [2, 0, 5, 7, 1, 6, 16, 13, 21, 21, 1, 28, 9, 9, 49, 37, 49, 33, 33, 60, 16, 40, 5, 59, 22, 22, 10, 33, 52, 79, 114, 50, 119, 112] 

Check Residues: [2, 0, 5, 7, 1, 6, 16, 13, 21, 21, 1, 28, 9, 9, 49, 37, 49, 33, 33, 60, 16, 40, 5, 59, 22, 22, 10, 33, 52, 79, 114, 50, 119, 112]



## CUDA parallel chinese remaindering example

In [3]:
import time
''' This example is for:
        2x
        x+1
        6x+2
'''

griddim = 2 # corresponds to the number of max degree polynomial
blockdim = 3 # the number of polynomials

a = np.array([2, 1, 6,0, 1, 2],dtype=np.int32 )
b = np.empty_like(a)
mvect = np.asarray([1,3,5]).astype(np.int32)
c = [0] + [EGCD(reduce(lambda x,y:x*y,mvect[0:k+1]),m)[1] for (k,m) in enumerate(mvect[1:])]
c= np.asarray(c).astype(np.int32)

CRA_kernel[griddim, blockdim](a,b,mvect,c) 

# You can check that these numbers are correct for mixed radix form
X = np.reshape(b,(griddim,blockdim))
G = reduce(lambda x,y: x*y,mvect)
print X
result = np.zeros(griddim)
for i in range(len(result)):
   
    result[i] = homer_scheme(mvect,X[i,:],0)%G
print result 

is_symmetric = False
m = mvect
Y = np.reshape(a,(2,3))
u = np.zeros(2)
G = reduce(lambda x,y: x*y,m)

for i,row in enumerate(Y):
    vv,M = MRC_alg(row,mvect,c)
    print vv
    temp = homer_scheme(mvect,vv,0)%G
    if is_symmetric:
        
        u[i] = symmetric_residue(temp,G)
    else:
        u[i] = temp
    

print u

   

[[2 2 4]
 [0 1 2]]
[ 1.  7.]
[2 2 4]
[0 1 2]
[ 1.  7.]


## CUDA Univariate polynomial multiplication

In [4]:
'Univariate Case'
from sympy.abc import x, y, u, v


#a = '2x^2  + 5'
#b = '1 x + 3'
a = '2x^2  + 5'
b = '1 x^1 + 3'

a = '7 x^1 + 5'
b = '2 x^1 - 3'

a_sym = Poly('x**2 + x + 5',x)
b_sym = Poly('x**2 + 5*x - 3',x)


a_sym = Poly('7*x + 5',x)
b_sym = Poly('2*x - 3',x)

#a = '7 x + 5'
#b = '2 x - 3'




aCoef,aVars = parsePolynomialString(a)
bCoef,bVars = parsePolynomialString(b)
aDegree = max([np.sum(_) for _ in buildDegrees(aVars)])
bDegree = max([np.sum(_) for _ in buildDegrees(bVars)])
cDegree = aDegree+bDegree
maxDegree = max(aDegree,bDegree)

aCoef = addZeros(aVars,aCoef,maxDegree)
bCoef = addZeros(bVars,bCoef,maxDegree)

#aCoef = [2,0,5]
#bCoef = [0,1,3]

''' setup '''
M = 2*int(max(np.abs(aCoef)))*int(max(np.abs(bCoef)) )
M = 2*M
print M
#Calculate homomorphisms
# syntax: x_'i' where i corresponds to mod reduction and x is the polynomial in question i.e (a,b)
# for this example we use m = 5,7
m = get_mod_primes(100,M)
m = m.tolist()
#m = [3,5,7,11]
is_symmetric = True
''' reduce polynomials'''
ab = np.hstack([np.hstack([np.mod(_Coef[::-1],mi) for mi in m]) for _Coef in [aCoef,bCoef]])

''' Set up CUDA parameters'''
#a = np.hstack([a_5[::-1],a_7[::-1]])
bpg = 50
tpb = 16
deg = cDegree
k=len(m)
n=deg+1 # the number of evaluation points
l=2  # the number of polies
alpha_os = 0
alphas = np.arange(n)+alpha_os
d_test = aDegree+1
N = d_test*k*n*l

c = np.empty(N,dtype=np.int32)

evaluate_polynomial[bpg,tpb](ab,c,n,d_test,k,l,alpha_os)

'''Reconstruct and interpolate'''
X = np.reshape(c,((N/d_test),d_test))
X = np.sum(X,axis=1)
X = np.reshape(X,(k*n,l),order='F')
X  = np.product(X,axis=1)
X = np.reshape(X,(k,n)) # final 

from sympy.polys.polyfuncs import interpolate
from sympy.abc import x

Y = np.empty_like(X)
for i,row in enumerate(X):
    Y[i,:] = newton_interp_univariate(x=alphas,b=row,d=deg)%m[i]
    Y[i,:] = Y[i,:] #[::-1]
    #Y[i,:] = Poly(interpolate(row,x)).all_coeffs()
   
gamma = [0] + [EGCD(reduce(lambda x,y:x*y,m[0:k_+1]),mi)[1] for (k_,mi) in enumerate(m[1:])]
d=2
u = np.zeros(deg+1)

for i,row in enumerate(Y.T):
    vv,M = MRC_alg(row,m,gamma)
    vv = vv.tolist()
    G = reduce(lambda x,y: x*y,m)
    #pdb.set_trace()
    temp = homer_scheme(m,vv,0)%G
    if is_symmetric:
        u[i] = symmetric_residue(temp,G)
    else:
        u[i] = temp
    
print "The coefficients are {}".format(u)

var_list = ['{}x^{}'.format(int(cf),p) for cf,p in zip(u,range(len(u))[::-1])]
print "The full poly is: {}".format(reduce(lambda x,y:x+' '+y,var_list))
print "Sympy's Solution is: {}".format(a_sym.mul(b_sym))

84
The coefficients are [ 14. -11. -15.]
The full poly is: 14x^2 -11x^1 -15x^0
Sympy's Solution is: Poly(14*x**2 - 11*x - 15, x, domain='ZZ')
